Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
#To not use GPU, on macbook, GPU memory of 512 MB is too little for most apps.
config = tf.ConfigProto(device_count = {'GPU': 0} )

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  #One hot mapping
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


### SGD with NN : 2 hidden layers

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000
hidden_nodes_layer1 = 500
hidden_nodes_layer2 = 500
learning_rate = 0.1
batch_size = 345

def compute_logits_2layer(dataset,h1_wts,h1_bias,h2_wts,h2_bias,o_wts,o_bias):
    h1_linear = tf.matmul(dataset, h1_wts) + h1_bias
    h1_out = tf.nn.relu(h1_linear)
    h2_linear = tf.matmul(h1_out,h2_wts) + h2_bias
    h2_out = tf.nn.relu(h2_linear)
    logits = tf.matmul(h2_out, o_wts) + o_bias
    return logits

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  h_weights_layer1 = tf.Variable(
    tf.random_normal(shape=[image_size * image_size, hidden_nodes_layer1],stddev=0.01))

  h_biases_layer1 = tf.Variable(tf.zeros([hidden_nodes_layer1]))

  h_weights_layer2 = tf.Variable(
    tf.random_normal(shape=[hidden_nodes_layer1, hidden_nodes_layer2],stddev=0.01))

  h_biases_layer2 = tf.Variable(tf.zeros([hidden_nodes_layer2]))
  
  o_weights = tf.Variable(
    tf.random_normal(shape=[hidden_nodes_layer2, num_labels],stddev=0.01))

  o_biases = tf.Variable(tf.zeros([num_labels]))
    
  tf.summary.histogram("h_wts_layer1",h_weights_layer1)
  tf.summary.histogram("h_bias_layer1",h_biases_layer1)
  tf.summary.histogram("h_wts_layer2",h_weights_layer2)
  tf.summary.histogram("h_bias_layer2",h_biases_layer2)
  tf.summary.histogram("o_wts",o_weights)
  tf.summary.histogram("o_bias",o_biases)

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = compute_logits_2layer(tf_train_dataset, h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  tf.summary.scalar('loss',loss)

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  #optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf.nn.softmax( compute_logits_2layer(tf_valid_dataset,h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases) )
    
  test_prediction = tf.nn.softmax( compute_logits_2layer(tf_test_dataset,h_weights_layer1, h_biases_layer1,h_weights_layer2, h_biases_layer2,
                          o_weights,o_biases) )

#### Run it

In [5]:
%%timeit -n 1 -r 1

num_steps = 12501

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph,config=config) as session:
    
  writer = tf.summary.FileWriter("./logs/",session.graph)
  merged = tf.summary.merge_all()
    
  tf.global_variables_initializer().run()
  #tf.initialize_all_variables().run()
    
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions,summary = session.run(
      [optimizer, loss, train_prediction , merged], feed_dict=feed_dict)
    
    writer.add_summary(summary, step)  # Write summary
        
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.301794
Minibatch accuracy: 12.8%
Validation accuracy: 14.8%
Test accuracy: 15.0%
Minibatch loss at step 500: 0.552093
Minibatch accuracy: 83.5%
Validation accuracy: 83.3%
Test accuracy: 89.3%
Minibatch loss at step 1000: 0.559035
Minibatch accuracy: 83.8%
Validation accuracy: 84.9%
Test accuracy: 91.0%
Minibatch loss at step 1500: 0.372819
Minibatch accuracy: 89.3%
Validation accuracy: 86.1%
Test accuracy: 92.0%
Minibatch loss at step 2000: 0.434740
Minibatch accuracy: 87.8%
Validation accuracy: 87.3%
Test accuracy: 92.7%
Minibatch loss at step 2500: 0.329490
Minibatch accuracy: 88.7%
Validation accuracy: 87.6%
Test accuracy: 93.3%
Minibatch loss at step 3000: 0.380052
Minibatch accuracy: 89.3%
Validation accuracy: 87.7%
Test accuracy: 93.4%
Minibatch loss at step 3500: 0.360140
Minibatch accuracy: 89.6%
Validation accuracy: 88.7%
Test accuracy: 94.1%
Minibatch loss at step 4000: 0.367392
Minibatch accuracy: 88.4%
Validation accuracy: 89.1%
Test 